### agenda
- intro
- logic
- code
- example

### intro

who thought work could be interesting? (kidding) <br>
one of the projects was to find intersections of cables along the road given the coordinates <br>
the business question was to determine if it is more profitable to buy another company's cable network rather than constructing own network <br>
the algorithm below finds intersection, coverage of layers in meters, and critically i would say it is a good solution but not the perfect one

### logic

in general, my aim was to divide each layer (cables around the road that we want to compare) into some intervals <br>
then i would find the distances between those intervals and if they were less than some threshold we could say the layers intersect <br>
to make us believe into the solution, visual part is needed otherwise we just rely on  <u> the best source </u>
<div style="text-align: center;">
    <img src="https://raw.githubusercontent.com/dsergazyyev/intersections/master/trustmebro.jpg" alt="examplepy" width="200" height="200">
</div>

### code

the best way to solve a problem is to break it down into small pieces (i'm sure you heard this a million times) <br>
first, we need to calculate the distance between two coordinates <br>
thanks to gpt to provide a ready to go (don't know the meaning but sounds good) function

In [1]:
# distance between two coordinates

from geopy.distance import geodesic

def distance_cc(coordinate1, coordinate2):
    
    return geodesic(coordinate1, coordinate2).meters

the function below creates intervals depending on how close do you want the coordinates to be <br>
to handle a case where you already have some coordinates that are located not too far there is some if statement <br>

In [2]:
# create intervals along the road

def create_intervals(road, interval = 100):
    intervals = []
    for i in range(len(road) - 1):
        currentc = road[i]
        nextc = road[i + 1]

        # calculate the distance between consecutive points
        distance = distance_cc(currentc, nextc)
    
        # check if distance is less than interval
        if distance < interval:
            if currentc not in intervals:
                intervals.append(currentc)
            if nextc not in intervals:
                intervals.append(nextc)
        else:
            # calculate the number of intervals needed
            num_intervals = int(distance / interval)

            # generate points along the line
            for j in range(num_intervals + 1):
                alpha = j / num_intervals
                point = (currentc[0] + alpha * (nextc[0] - currentc[0]), currentc[1] + alpha * (nextc[1] - currentc[1]))

                if point not in intervals:
                    intervals.append(point)
    return intervals

the middle points are ingredients of intersection line, so to make it more fancy i separated the function (thought it doesn't play a big role)

In [3]:
def find_middle(coordinate1, coordinate2):
    return ((coordinate1[0] + coordinate2[0])/2, (coordinate1[1] + coordinate2[1])/2)

in short, intersections function generates a list of middle points <br>
you can choose threshold of how close cables could be to find intersection <br>
by default it is 50 meters 

In [4]:
def intersections(road1, road2, threshold = 50):
    # extended roads
    road1_extended = create_intervals(road1)
    road2_extended = create_intervals(road2)
    
    mid = []
    # inters
    for p1 in road1_extended:
        for p2 in road2_extended:
            dist = distance_cc(p1, p2)
            
            if dist < threshold:
                midp = find_middle(p1, p2)
                mid.append(midp)
            
    return mid

to compare the lengths of cables (in our case) the function below calculates the coverage of a layer (or cable)

In [5]:
def len_result(road):
    # calculate length of cable
    sum = 0
    for i in range(len(road)-1):
        sum += distance_cc(road[i], road[i+1])
    return int(sum)

folium is my new bestfriend (of course after gpt) because it makes the work visible 

In [6]:
import folium
from IPython.display import display

def plot_map(layer1, layer2, intersections):

    # centered map
    map_center = [
        (sum(x[0] for x in layer1) + 
         sum(y[0] for y in layer2) +
         sum(z[0] for z in intersections)) / 
        (len(layer1) + len(layer2) + len(intersections)),
        (sum(x[1] for x in layer1) + 
         sum(y[1] for y in layer2) +
         sum(z[1] for z in intersections)) / 
        (len(layer1) + len(layer2) + len(intersections)) ]
    
    my_map = folium.Map(location=map_center, zoom_start=10)
  
    l1 = folium.FeatureGroup(name='Layer 1')
    l2 = folium.FeatureGroup(name='Layer 2')
    l3 = folium.FeatureGroup(name='Intersection')
    
    # lines
    folium.PolyLine(locations=layer1, color='red').add_to(l1)
    folium.PolyLine(locations=layer2, color='blue').add_to(l2)   
    folium.PolyLine(locations=intersections, color='black').add_to(l3)
    
    l1.add_to(my_map)
    l2.add_to(my_map)
    l3.add_to(my_map)
    
    folium.LayerControl().add_to(my_map)

    return my_map

### example

In [7]:
r1 = [
    (37.7749, -122.4194),
    (37.7755, -122.4185),
    (37.7761, -122.4176),
    (37.7770, -122.4168),
    (37.7780, -122.4158),
    (37.7790, -122.4145),
    (37.7800, -122.4130),
    (37.7810, -122.4115),
    (37.7820, -122.4100),
    (37.7830, -122.4085),
    (37.7840, -122.4070),
    (37.7850, -122.4055),
    (37.7860, -122.4040),
    (37.7870, -122.4030),
    (37.7880, -122.4020),
    (37.7890, -122.4010),
    (37.7900, -122.4000),
    (37.7910, -122.3990),
    (37.7920, -122.3980),
    (37.7930, -122.3970)]


In [8]:
r2 = [
    (37.7749, -122.4194),
    (37.7755, -122.418),
    (37.7761, -122.4166),
    (37.777, -122.415),
    (37.778, -122.4135),
    (37.779, -122.412),
    (37.78, -122.4105),
    (37.781, -122.409),
    (37.782, -122.4075),
    (37.783, -122.406),
    (37.784, -122.4045),
    (37.784, -122.402),
    (37.784, -122.3995),
    (37.784, -122.3975),
    (37.784, -122.3955),
    (37.784, -122.3935)]

In [9]:
inters = intersections(r1, r2, 150)

In [10]:
plot_map(r1, r2, inters)

In [11]:
print(f'the length of red line is {len_result(r1)} meters')
print(f'the length of blue line is {len_result(r2)} meters')
print(f'the length of black line (intersection) is {len_result(inters)} meters')

the length of red line is 2830 meters
the length of blue line is 2630 meters
the length of black line (intersection) is 1702 meters


other calculations and analysis depend on you: <br>
the percent of coverage, whether red line is your company and you want to compare it to other ones which would represent blue lines <br>
have fun and i wish you to implement a better solution :)